# Import Dependencies

In [ ]:
import openai

# What is OpenAI API?

The OpenAI API refers to the application programming interface provided by OpenAI, a leading artificial intelligence research lab. The OpenAI API allows developers to access and integrate OpenAI's powerful language models into their own applications, products, or services.

One notable model offered through the OpenAI API is GPT (Generative Pre-trained Transformer). GPT models are designed for natural language processing tasks, such as text generation, completion, translation, summarization, and more. The API provides a way for developers to interact with these models and leverage their capabilities without having to train the models from scratch.

Keep in mind that the specifics of the OpenAI API, including its features and offerings, may have evolved since my last update in January 2022. Therefore, it's a good idea to check the official OpenAI website or other reliable sources for the most up-to-date information on the OpenAI API.


# Generate OPEN API Key

In [ ]:
Myapi = "sk-"

In [ ]:
openai.api_key = Myapi

In [ ]:
All_Models = openai.models.list()

In [ ]:
list(All_Models)

In [ ]:
import pandas as pd

In [ ]:
pd.DataFrame(list(All_Models), columns = ["ID","Created", "Object","Owned_By"])

## Code of OpenAI Chat

In [ ]:
from openai import OpenAI
client = OpenAI()

response = client.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=[
    {
      "role": "user",
      "content": "How I can Make Money?"
    }
  ],
  temperature=1,
  max_tokens=256,
  top_p=1,
  frequency_penalty=0,
  presence_penalty=0
)

## This is where you make a request to the OpenAI API for generating a chat completion. Let's break down the parameters:

    model="gpt-3.5-turbo": Specifies the language model to be used, in this case, GPT-3.5-turbo.

    messages: A list of message objects representing the conversation. In this example, there is only one message from the user: "How I can Make Money?".

    temperature=1: A parameter controlling the randomness of the model's output. Higher values like 1 make the output more random, while lower values like 0.2 make it more focused and deterministic.

    max_tokens=256: Specifies the maximum number of tokens (words or characters) in the generated response.

    top_p=1: Top-p (nucleus) sampling probability. Setting it to 1 means the model considers the entire probability distribution for sampling.

    frequency_penalty=0: Controls the penalty for using frequent tokens. A higher value will make the model more creative, while a lower value will make it more conservative.

    presence_penalty=0: Controls the penalty for using new tokens. Similar to frequency_penalty, but for unseen tokens.

# Langchain for Generative AI

Function Calling : Learn how to connect Large Language Models to external tools.

Example invoking multiple function calls in one response

In [ ]:
from openai import OpenAI
import json

client = OpenAI()

# Example dummy function hard coded to return the same weather
# In production, this could be your backend API or an external API
def get_current_weather(location, unit="fahrenheit"):
    """Get the current weather in a given location"""
    if "tokyo" in location.lower():
        return json.dumps({"location": "Tokyo", "temperature": "10", "unit": unit})
    elif "san francisco" in location.lower():
        return json.dumps({"location": "San Francisco", "temperature": "72", "unit": unit})
    elif "paris" in location.lower():
        return json.dumps({"location": "Paris", "temperature": "22", "unit": unit})
    else:
        return json.dumps({"location": location, "temperature": "unknown"})

def run_conversation():
    # Step 1: send the conversation and available functions to the model
    messages = [{"role": "user", "content": "What's the weather like in San Francisco, Tokyo, and Paris?"}]
    tools = [
        {
            "type": "function",
            "function": {
                "name": "get_current_weather",
                "description": "Get the current weather in a given location",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "location": {
                            "type": "string",
                            "description": "The city and state, e.g. San Francisco, CA",
                        },
                        "unit": {"type": "string", "enum": ["celsius", "fahrenheit"]},
                    },
                    "required": ["location"],
                },
            },
        }
    ]
    response = client.chat.completions.create(
        model="gpt-3.5-turbo-1106",
        messages=messages,
        tools=tools,
        tool_choice="auto",  # auto is default, but we'll be explicit
    )
    response_message = response.choices[0].message
    tool_calls = response_message.tool_calls
    # Step 2: check if the model wanted to call a function
    if tool_calls:
        # Step 3: call the function
        # Note: the JSON response may not always be valid; be sure to handle errors
        available_functions = {
            "get_current_weather": get_current_weather,
        }  # only one function in this example, but you can have multiple
        messages.append(response_message)  # extend conversation with assistant's reply
        # Step 4: send the info for each function call and function response to the model
        for tool_call in tool_calls:
            function_name = tool_call.function.name
            function_to_call = available_functions[function_name]
            function_args = json.loads(tool_call.function.arguments)
            function_response = function_to_call(
                location=function_args.get("location"),
                unit=function_args.get("unit"),
            )
            messages.append(
                {
                    "tool_call_id": tool_call.id,
                    "role": "tool",
                    "name": function_name,
                    "content": function_response,
                }
            )  # extend conversation with function response
        second_response = client.chat.completions.create(
            model="gpt-3.5-turbo-1106",
            messages=messages,
        )  # get a new response from the model where it can see the function response
        return second_response
print(run_conversation())

In [ ]:
import langchain

In [ ]:
from langchain.llms import OpenAI

In [ ]:
client = OpenAI(openai_api_key=Myapi)

In [ ]:
# Zero short prompting
prompt = "Can you tell me total number of country in Aisa?"

In [ ]:
client.predict(prompt)